<a href="https://colab.research.google.com/github/Ravindra1972/Anaytics-in-finance-using-Python/blob/main/CAPM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm


In [ ]:
!pip install getFamaFrenchFactors
import getFamaFrenchFactors as gff

In [ ]:
ticker = "AMZN"
stock_data = yf.download(ticker, start="2016-06-30", end="2024-07-20")

stock_prices = stock_data['Adj Close']
stock_prices = stock_prices.resample("1M").last()
returns = stock_prices.pct_change()
returns = returns.dropna()
returns.name = "Asset"

[*********************100%%**********************]  1 of 1 completed


In [ ]:
ff3_monthly = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)
ff_data = ff3_monthly.merge(returns, on='Date')
rf = ff_data['RF'].mean()
market_premium = ff_data['Mkt-RF'].mean()

In [ ]:
X = ff_data['Mkt-RF']
ff_data['Asset-RF'] = ff_data['Asset'] - ff_data['RF']
y = ff_data['Asset-RF']
c = sm.add_constant(X)
capm_model = sm.OLS(y, c)
result = capm_model.fit()
print(result.summary())
intercept, beta = result.params

                            OLS Regression Results                            
Dep. Variable:               Asset-RF   R-squared:                       0.427
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     69.33
Date:                Sat, 20 Jul 2024   Prob (F-statistic):           7.02e-13
Time:                        14:26:43   Log-Likelihood:                 121.89
No. Observations:                  95   AIC:                            -239.8
Df Residuals:                      93   BIC:                            -234.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0064      0.007      0.892      0.3

In [ ]:
expected_return = rf + beta*market_premium
print("Expected monthly returns: " + str(expected_return))
yearly_return = expected_return * 12
print("Expected yearly returns: " + str(yearly_return))

Expected monthly returns: 0.014432362599361035
Expected yearly returns: 0.17318835119233242
